\
            # 49. 工程基础：Git（版本控制）（Git Fundamentals）

            目标：把 Git 的“对象模型 + 常用命令 + 回滚/排错”讲清楚，形成稳定工作流。
本章提供一个可运行的最小仓库演示（如果本机安装了 git）。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 命令行基础（cd/ls）更佳


## 知识点地图

- 1. Git 是什么：用快照管理历史
- 2. 三区域模型：工作区 / 暂存区 / 提交
- 3. 日常命令速记：status/add/commit/log/diff
- 4. 分支与合并：merge vs rebase
- 5. 撤销与回滚：restore/reset/revert + reflog
- 6. 远端协作：clone/fetch/pull/push 与冲突处理
- 7. 可运行：创建一个最小仓库并演示提交历史（可选）


## 自检清单（学完打勾）

- [ ] 理解工作区/暂存区/提交（working tree/index/commit）
- [ ] 会用 status/add/commit/log/diff 进行日常提交
- [ ] 理解分支与合并，知道 merge 与 rebase 的差异
- [ ] 会撤销：restore/reset/revert，并会用 reflog 找回
- [ ] 会与远端协作：fetch/pull/push，理解冲突解决流程


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：Git 是什么：用快照管理历史

Git 的核心不是“存 diff”，而是存一组对象快照：
- blob：文件内容
- tree：目录结构
- commit：指向 tree + 父 commit + 元数据

分支本质：一个可移动的指针（指向某个 commit）。


## 知识点 2：三区域模型：工作区 / 暂存区 / 提交

- working tree：你正在编辑的文件
- index（staging）：准备提交的内容
- commit：历史记录

常用命令：
- `git status` 看当前状态
- `git add` 把变更放入暂存
- `git commit` 形成一次可回溯的历史点


## 知识点 3：日常命令速记：status/add/commit/log/diff

- `git diff`：工作区 vs 暂存区
- `git diff --staged`：暂存区 vs 上次提交
- `git log --oneline --graph --decorate --all`：看分支结构

好习惯：
- 小步提交、信息清晰、一次提交只做一件事。


## 知识点 4：分支与合并：merge vs rebase

- merge：保留分叉历史（可能产生 merge commit），适合团队协作可追踪。
- rebase：把分支“挪到”新基底，历史更线性；适合个人分支整理提交。

原则：
- 公共分支上慎用 rebase（会改历史）。


## 知识点 5：撤销与回滚：restore/reset/revert + reflog

- `git restore <file>`：丢弃工作区修改
- `git reset`：移动 HEAD/分支指针（可改历史）
- `git revert <commit>`：生成一个“反向提交”（不改历史，适合公共分支）
- `git reflog`：找回“你曾经指向过的 commit”（救命工具）


## 知识点 6：远端协作：clone/fetch/pull/push 与冲突处理

- `fetch`：只拉取远端更新，不合并
- `pull`：fetch + merge/rebase
- `push`：推送本地分支到远端

冲突处理流程：
1) 定位冲突文件 2) 手工选择保留内容 3) add 4) commit


## 知识点 7：可运行：创建一个最小仓库并演示提交历史（可选）

如果你电脑装了 git，本段会在 `_nb_artifacts/git_demo` 创建一个临时仓库并跑几条命令。
如果没装，会输出提示。


In [ ]:
import os
import shutil
import subprocess
from pathlib import Path


def run(cmd, cwd):
    p = subprocess.run(cmd, cwd=cwd, text=True, capture_output=True)
    return p.returncode, p.stdout.strip(), p.stderr.strip()


git = shutil.which('git')
if not git:
    print('git not found. Please install Git and ensure it is in PATH.')
else:
    repo = Path('_nb_artifacts') / 'git_demo'
    if repo.exists():
        # 清理旧演示目录（谨慎：仅删 _nb_artifacts 下的 demo）
        for p in repo.rglob('*'):
            if p.is_file():
                p.unlink()
        for p in sorted([p for p in repo.rglob('*') if p.is_dir()], reverse=True):
            p.rmdir()
        repo.rmdir()

    repo.mkdir(parents=True)
    print('repo:', repo.resolve())

    print(run([git, 'init'], repo)[1])
    (repo / 'a.txt').write_text('hello\n', encoding='utf-8')
    run([git, 'add', 'a.txt'], repo)
    run([git, 'commit', '-m', 'init'], repo)

    (repo / 'a.txt').write_text('hello\nworld\n', encoding='utf-8')
    run([git, 'add', 'a.txt'], repo)
    run([git, 'commit', '-m', 'add world'], repo)

    code, out, err = run([git, 'log', '--oneline', '--decorate'], repo)
    print(out)


## 常见坑

- 把所有改动塞进一个 commit：难 review、难回滚
- 在公共分支 rebase：改历史导致他人分支混乱
- 误用 reset：把历史“弄没了”（其实通常还能用 reflog 找回）
- 只会 pull 不会 fetch：看不清远端状态就合并


## 综合小案例：模拟一次冲突并解决（写下步骤）

场景：两个人同时改同一行代码。
请写出你会怎么做：
1) 拉取/更新分支
2) 产生冲突
3) 打开文件解决冲突标记
4) add + commit
5) push 并发起 PR/MR


In [ ]:
# 练习建议：在本机真实建一个 repo，开两个分支改同一行制造冲突。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- 分支在 Git 里本质是什么？
- merge 与 rebase 的差异是什么？什么时候用哪个？
- reset 与 revert 的差异是什么？为什么公共分支更推荐 revert？
- reflog 能解决什么问题？


## 练习题（建议写代码）

- 把一个“混乱提交历史”的分支用交互式 rebase 整理成 3 个有意义的提交（了解）。
- 为项目写一份 `.gitignore`（Python + 前端 + OS）。
- 为团队制定提交规范（commit message 规范 + 分支命名 + PR 流程）。
